<a href="https://colab.research.google.com/github/iholthey/Google-Sheets/blob/main/Dropbox_Analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Benötigte Bibliotheken installieren und laden
!pip install dropbox
import dropbox
import pandas as pd
import gspread
from google.colab import auth, drive
from google.auth import default

# --- SCHRITT A: Authentifizierung ---

# Google Authentifizierung (für Sheets)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Dropbox Authentifizierung
# Du benötigst einen App-Key von https://www.dropbox.com/developers/apps
APP_KEY = 'tk2r74ecpw2ssn1s'
APP_SECRET = 'jglx13uch33kar8'

auth_flow = dropbox.DropboxOAuth2FlowNoRedirect(APP_KEY, APP_SECRET)
authorize_url = auth_flow.start()

print("1. Besuche diese URL, um dich bei Dropbox anzumelden: \n", authorize_url)
print("2. Kopiere den Autorisierungscode hier rein:")
auth_code = input("Code eingeben: ").strip()

try:
    oauth_result = auth_flow.finish(auth_code)
    dbx = dropbox.Dropbox(oauth_result.access_token)
    print("Erfolgreich bei Dropbox angemeldet!")
except Exception as e:
    print("Fehler bei der Anmeldung:", e)

# --- SCHRITT B: Dropbox Daten auslesen (Limit 10) ---

file_list = []
print("Lese Dateien aus Dropbox...")

# Ruft die Dateiliste ab
result = dbx.files_list_folder('')

for entry in result.entries:
    if isinstance(entry, dropbox.files.FileMetadata):
        # Sammle verfügbare Informationen
        file_list.append({
            'Name': entry.name,
            'Pfad': entry.path_display,
            'Größe (Bytes)': entry.size,
            'Letzte Änderung': str(entry.client_modified),
            'ID': entry.id
        })

    # Test-Abbruch nach 10 Dateien
    if len(file_list) >= 10:
        break

df = pd.DataFrame(file_list)

# --- SCHRITT C: In Google Sheets speichern ---

sheet_name = "Dropbox Analyse"
try:
    # Prüfen, ob das Sheet schon existiert, sonst neu erstellen
    sh = gc.create(sheet_name)
    print(f"Neues Sheet '{sheet_name}' wurde erstellt.")
except:
    sh = gc.open(sheet_name)
    print(f"Existierendes Sheet '{sheet_name}' geöffnet.")

# Lasche "Dateien" erstellen oder wählen
try:
    worksheet = sh.add_worksheet(title="Dateien", rows="100", cols="20")
except:
    worksheet = sh.worksheet("Dateien")

# Daten in das Sheet schreiben
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

print("Fertig! Die ersten 10 Dateien wurden in 'Dropbox Analyse' gespeichert.")

1. Besuche diese URL, um dich bei Dropbox anzumelden: 
 https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=tk2r74ecpw2ssn1
2. Kopiere den Autorisierungscode hier rein:
Code eingeben: LdEOMdBuxkQAAAAAAAGYWI4_jRkfelCeyBOKAo9NJ1Y
Erfolgreich bei Dropbox angemeldet!
Lese Dateien aus Dropbox...
Neues Sheet 'Dropbox Analyse' wurde erstellt.
Fertig! Die ersten 10 Dateien wurden in 'Dropbox Analyse' gespeichert.


# Task
Explanation regarding authentication:
The authentication code for Dropbox (and similar services) is sometimes requested repeatedly because access tokens have a limited lifespan for security reasons. If your Colab session restarts, if the token expires, or if you revoke access, you will need to re-authenticate. However, once successfully authenticated within an active Colab session, you generally won't be asked for the code again unless one of these conditions is met.

Now, let's proceed with updating the existing code to recursively retrieve up to 50 Dropbox files with enhanced metadata and save them to Google Sheets.

**Original User Task**: Adapt the existing code in cell `KPZSCnCcM-Dz` to recursively retrieve Dropbox files (including subfolders) and collect the metadata 'Dateiname mit Suffix', 'Pfad', 'Größe (Bytes)', 'Client letzte Änderung', 'Server letzte Änderung' (interpretiert als 'Erstelldatum'), and 'ID' for the first 50 entries. Then, save these extended file metadata into the 'Dateien'-Tab of the 'Dropbox Analyse'-Sheet in Google Sheets, updating the column headers to reflect the new fields.

## Erläuterung der Authentifizierung

### Subtask:
Erklären, warum der Authentifizierungscode manchmal wiederholt angefordert wird (z.B. nach Session-Neustart oder wenn der Token abläuft), aber bei korrekter erstmaliger Eingabe normalerweise nicht innerhalb einer aktiven Colab-Sitzung.


## Dropbox-Dateien rekursiv abrufen und Metadaten sammeln

### Subtask:
Den vorhandenen Code in Zelle `KPZSCnCcM-Dz` anpassen, um Dropbox-Dateien rekursiv (einschließlich Unterordner) abzurufen und die Metadaten 'Dateiname mit Suffix', 'Pfad', 'Größe (Bytes)', 'Client letzte Änderung', 'Server letzte Änderung' und 'ID' für die ersten 50 Einträge zu sammeln.


**Reasoning**:
The subtask requires modifying the existing code cell `KPZSCnCcM-Dz` to recursively list Dropbox files, collect specific metadata for the first 50 entries, and handle API pagination. I will replace the relevant section of the cell with the updated logic.



In [11]:
# 1. Benötigte Bibliotheken installieren und laden
!pip install dropbox
import dropbox
import pandas as pd
import gspread
from google.colab import auth, drive
from google.auth import default

# --- SCHRITT A: Authentifizierung ---

# Google Authentifizierung (für Sheets)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Dropbox Authentifizierung
# Du benötigst einen App-Key von https://www.dropbox.com/developers/apps
APP_KEY = 'tk2r74ecpw2ssn1s'
APP_SECRET = 'jglx13uch33kar8'

auth_flow = dropbox.DropboxOAuth2FlowNoRedirect(APP_KEY, APP_SECRET)
authorize_url = auth_flow.start()

print("1. Besuche diese URL, um dich bei Dropbox anzumelden: \n", authorize_url)
print("2. Kopiere den Autorisierungscode hier rein:")
auth_code = input("Code eingeben: ").strip()

try:
    oauth_result = auth_flow.finish(auth_code)
    dbx = dropbox.Dropbox(oauth_result.access_token)
    print("Erfolgreich bei Dropbox angemeldet!")
except Exception as e:
    print("Fehler bei der Anmeldung:", e)

# --- SCHRITT B: Dropbox Daten rekursiv auslesen (Limit 50) ---

file_list = []
print("Lese Dateien aus Dropbox rekursiv...")

# Ruft die Dateiliste ab (rekursiv)
result = dbx.files_list_folder(path='', recursive=True)

while True:
    for entry in result.entries:
        if isinstance(entry, dropbox.files.FileMetadata):
            # Sammle verfügbare Informationen
            file_list.append({
                'Dateiname mit Suffix': entry.name,
                'Pfad': entry.path_display,
                'Größe (Bytes)': entry.size,
                'Client letzte Änderung': str(entry.client_modified),
                'Server letzte Änderung': str(entry.server_modified), # Genutzt als 'Erstelldatum'
                'ID': entry.id
            })

        # Test-Abbruch nach 50 Dateien
        if len(file_list) >= 50:
            break

    if len(file_list) >= 50 or not result.has_more:
        break

    result = dbx.files_list_folder_continue(result.cursor)

df = pd.DataFrame(file_list)

# --- SCHRITT C: In Google Sheets speichern ---

sheet_name = "Dropbox Analyse"
try:
    # Prüfen, ob das Sheet schon existiert, sonst neu erstellen
    sh = gc.create(sheet_name)
    print(f"Neues Sheet '{sheet_name}' wurde erstellt.")
except:
    sh = gc.open(sheet_name)
    print(f"Existierendes Sheet '{sheet_name}' geöffnet.")

# Lasche "Dateien" erstellen oder wählen
try:
    worksheet = sh.add_worksheet(title="Dateien", rows="100", cols="20")
except:
    worksheet = sh.worksheet("Dateien")

# Daten in das Sheet schreiben
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

print("Fertig! Die ersten 50 Dateien (rekursiv) wurden in 'Dropbox Analyse' gespeichert.")

1. Besuche diese URL, um dich bei Dropbox anzumelden: 
 https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=tk2r74ecpw2ssn1s
2. Kopiere den Autorisierungscode hier rein:
Code eingeben: LdEOMdBuxkQAAAAAAAGYWI4_jRkfelCeyBOKAo9NJ1Y
Fehler bei der Anmeldung: 400 Client Error: Bad Request for url: https://api.dropboxapi.com/oauth2/token
Lese Dateien aus Dropbox rekursiv...
Neues Sheet 'Dropbox Analyse' wurde erstellt.
Fertig! Die ersten 50 Dateien (rekursiv) wurden in 'Dropbox Analyse' gespeichert.
